<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment1/blob/main/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing necessary libraries.

In [2]:
import keras
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import copy 


Installing and importing wandb

In [3]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 143 kB 56.0 MB/s 
     |████████████████████████████████| 180 kB 38.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


Preparing dataset

In [1]:
def prepdata():
  # Loading data
  (train_x,train_y),(test_x,test_y)=fashion_mnist.load_data()
  # Defining labels for data
  labels=['T-shirt/top','Trouser','Pullover','Dresss','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
  
  image_shape=train_x.shape[1]*train_x.shape[2]
  
  train_image_count=len(train_x)
  test_image_count=len(test_x)
  
  # Creating a matrix of image data 
  # each image is represented as a row by flattening the matrix: converting (60000,28,28) tensor to (60000,784) matrix
  x_train=np.zeros((train_image_count,image_shape))
  x_test=np.zeros((test_image_count,image_shape))
  
  for i in range(train_image_count):
    x_train[i]=(copy.deepcopy(train_x[i].flatten()))/255.0 # converting the images into grayscale
  for i in range(test_image_count):
    x_test[i]=(copy.deepcopy(test_x[i].flatten()))/255.0
  
  # Creating training and testing dataset,
  # with each row corresponding to an image and its associated label.
  train_data=np.array(list(zip(x_train,train_y)))
  test_data=np.array(list(zip(x_test,test_y)))

  # Creating validation dataset, with 10% of training data
  train_data_len=len(train_data)
  val_data_len=int(0.1*train_data_len)

  idx=np.random.choice(val_data_len,train_data_len,replace=False)

  val_data=train_data[idx]

  
  return(train_data,test_data,val_data,labels)
  

Plotting 1 sample image for each class in wandb